In [1]:
import pyfastcopy
import os
import shutil
import gc
import pandas as pd
import concurrent.futures
from tqdm import tqdm

def copy_images_batch(src_list, dest_list, batch_size=512):
    src_files=src_list
    dest_files=dest_list    
    with concurrent.futures.ThreadPoolExecutor() as exe:
        batch_tasks = []
        for i in tqdm(range(0, len(src_files), batch_size)):
            src_batch = src_files[i:i + batch_size]
            dest_batch = dest_files[i:i + batch_size]
            batch_tasks.extend([exe.submit(shutil.copy, src, dest) for src, dest in zip(src_batch, dest_batch)])
            # Wait for all tasks in the batch to complete before proceeding to the next batch
            _ = [task.result() for task in batch_tasks]
    return

def list_files_in_directory(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg','.json','.csv')):
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
            else:
                continue
    return file_paths

In [2]:
source_dir = r"D:\Camera_trapping\Guzzler_data\2023\CameraTrap\Ganga\Ganga1"
dest_dir = r"G:\Guzzler_data\2023\Segregated_guzzler_data2023\Ganga\Ganga1"

In [3]:
sub_directories=[os.path.join(source_dir, d) for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
if not sub_directories:
    sub_directories= [source_dir]
sub_directories

['D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\1',
 'D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\2',
 'D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\3',
 'D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\4',
 'D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\5',
 'D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\6',
 'D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\7',
 'D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\8',
 'D:\\Camera_trapping\\Guzzler_data\\2023\\CameraTrap\\Ganga\\Ganga1\\9']

In [4]:
### New
source_dir = r"J:\Camera_Trapping\Guzzler_data\2023\CameraTrap\SudasariACD\C2"
sub_directories=[os.path.join(source_dir, d) for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
if not sub_directories:
    sub_directories= [source_dir]
print(sub_directories)

for dir in sub_directories:
    dest_folder = os.path.join(dest_dir,"\\".join(dir.split("\\")[-3:]))
    if not os.path.isdir(dest_folder):
        os.makedirs(dest_folder, exist_ok = True)
        df=pd.DataFrame({"src_list" : list_files_in_directory(dir)})
        df["dest_list"] = dest_dir + "\\" + df["src_list"].apply(lambda x: "\\".join(x.split("\\")[-4: ]))
        copy_images_batch(df['src_list'],df['dest_list'], batch_size=512)
        gc.collect()
    elif len(os.listdir(dest_folder)) < len(os.listdir(dir)):
        df=pd.DataFrame({"src_list" : list_files_in_directory(dir)})
        df["dest_list"] = dest_dir + "\\" + df["src_list"].apply(lambda x: "\\".join(x.split("\\")[-4: ]))
        copy_images_batch(df['src_list'],df['dest_list'], batch_size=512)
        gc.collect()
    else:
        print(f"{dest_folder} already exists and is not being copied")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'J:\\Camera_Trapping\\Guzzler_data\\2023\\CameraTrap\\SudasariACD\\C2'